In [1]:
import pandas as pd

from tqdm import tqdm

import nltk

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("[\w']+").tokenize

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer().lemmatize


from nltk.stem import PorterStemmer
stemmer = PorterStemmer().stem

from nltk.corpus import stopwords
stopwords = stopwords.words('english')


from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import random

from datetime import date
from datetime import datetime, timedelta


In [2]:
#df = pd.read_csv('raw_data.csv')
#df = df.astype({'UserID':'str'})

#beforedf = len(df)

In [3]:
#for i in tqdm(range(len(df))):
#    df['Review'][i] = ' ' + str(df['Review'][i]) + ' ' 

In [4]:
#df.to_csv('raw_data(띄어쓰기+).csv')
df = pd.read_csv('raw_data(띄어쓰기+).csv')

In [5]:
for i in range(10):
    #df['Review'] = df['Review'].str.replace('é', 'e')
    df['Review'] = df['Review'].str.replace('pay 2 win', ' ptw ')
    df['Review'] = df['Review'].str.replace('pay to win', ' ptw ')
    df['Review'] = df['Review'].str.replace('p2w', ' ptw ')
    df['Review'] = df['Review'].str.replace('random number generator', ' rng ')
    df['Review'] = df['Review'].str.replace('real money trade', ' rmt ')
    df['Review'] = df['Review'].str.replace('cash shop', ' cashshop ')
    df['Review'] = df['Review'].str.replace('micro transaction', ' microtransaction ')
    
    df['Review'] = df['Review'].str.replace('path of exile', ' poe ')
    df['Review'] = df['Review'].str.replace('new world', ' newworld ')
    df['Review'] = df['Review'].str.replace(' animation ', ' cutscene ')
    df['Review'] = df['Review'].str.replace('world of warcraft', ' wow ')
    df['Review'] = df['Review'].str.replace('lost ark', ' lostark ')
    df['Review'] = df['Review'].str.replace('arpg', 'rpg')
    df['Review'] = df['Review'].str.replace('mmo rpg', ' mmorpg ')
    df['Review'] = df['Review'].str.replace(' gameplay ', ' game play ')
    df['Review'] = df['Review'].str.replace(' log out ', ' logout ')
    df['Review'] = df['Review'].str.replace(' log in ', ' login ')

    df['Review'] = df['Review'].str.replace(' msq ', ' main story quest ')
    #df['Review'] = df['Review'].str.replace(' npc ', ' non player character ')
    #df['Review'] = df['Review'].str.replace(' mob ', ' mobile object ')
    df['Review'] = df['Review'].str.replace(' cut scene ', ' cutscene ')
    df['Review'] = df['Review'].str.replace(' gender lock ', ' genderlock ')

* 스페인어 제외

In [6]:
spanish = ['å', 'ğ', 'м', ' del ', ' vittua ', ' resumo ', 'ů', 'и', 'ÿ', 'ï', ' lamas ', 'л', 'ч', 'ı', 'õ', 'ř', 'ń', 'ě', ' nie ', 'ň', 'ã', 'ť', 'ñ', ' qualcuno ', ' jogo ', 'ž', ' wenig ', ' kreat ', ' sich ', 'â', 'ď', ' esta ', 'ċ', 'è', 'ő', 'ặ', 'т', ' wenn ', 'ṃ', 'ẻ', 'ş', 'à', 'ẗ', ' soll ', ' walter ', ' ich ', 'я', 'ý', 'ä', ' juego ', ' dej ', ' muito ', 'д', 'ó', ' oldu ', 'í', 'č', 'ė', 'ṗ', 'ọ', 'ô', ' cie ', '½', 'ű', '¾', ' spiel ', 'á', 'ß', 'ľ', '¼', 'ú', 'ệ', 'ì', 'ʖ', 'é', 'ŕ', 'ṩ', 'ü', 'š', 'ö', ' gen ', ' certo ', 'ê', 'ù', ' jeu ', 'ṿ', 'ç', 'у']

x = len(df)
for i in spanish:
    df = df[~df['Review'].str.contains(i)]    
    
y = len(df)

print(f'{x-y}개의 스페인어 제외')

x = len(df)

ads = [' piece of candy ', ' impurities ', ' piola ', ' juego ', ' brush ', ' toothbrush ', ' xxxxxxx ', 'gggg', ' sugar ',' salt ', ' hog rider ', ' tsp ', ' eargasm ', ' red sun ', ' lemonade ', ' toothbursh ',' this game saved my ' ]

for i in ads:
    df = df[~df['Review'].str.contains(i)]    
    
y = len(df)


print(f'{x-y}개의 광고성 제외')

df["Review"] = df["Review"].str.replace(pat=r'[0-9]', repl=r' ', regex=True)
df["Review"] = df["Review"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
df["Review"] = df["Review"].str.replace(pat=r'[^a-z0-9]', repl=r' ', regex=True)
df["Review"] = df["Review"].str.replace(pat=r'[ㄱ-ㅣ가-힣]+', repl= r' ', regex=True)

x = len(df)
for i in spanish:
    df = df[~df['Review'].str.contains(i)]    
    
y = len(df)

print(f'{x-y}개의 스페인어 제외')

x = len(df)

for i in ads:
    df = df[~df['Review'].str.contains(i)]    
    
y = len(df)


print(f'{x-y}개의 광고성 제외')


648개의 스페인어 제외
269개의 광고성 제외
24개의 스페인어 제외
37개의 광고성 제외


* 광고성, 주제와 상관없는 라인들 제외

In [7]:
#for i in ['mobile']:
    #df2 = df[df['Review'].str.contains(i)]
    #print(len(df2['Review'].to_list()))
    #df2.to_csv(f'{i}.csv')

In [8]:
#df2 = df[df['Review'].str.contains(' piece of candy ')]
#len(df2['Review'].to_list())
#df2['Review'].to_list()

* 전처리(synonyms)

In [9]:
stops = ['next','small','fast', 'slow', 'old', 'new', 'long', 'short', 'must',
         'thousand', 'god','guy','man','woman','damn','either','min','hundred',
         'unless', 'plenty','quite','minute', 'less','year','second','ags','month','ton',
         'try', 'want', 'see','need','big','month', 'hour', 'hrs','name',
         'review', 'look','think','feel','say','know', 'lailai', 'felt', 
         'come', 'back','first','little','day','enough','rather','lostark',
         'keep','year','put','let', 'thank', 'stuff', 'people', 'else', 'wanna','anyone', 
         'ark', 'lost', 'lose', 'make','nothing','meh','better',
         'like','amount', 'hour','go','give','get','take','play',
         'super', 'gameplay','sometime', 
         'positive', 'negative', 'gonna', 'smack', 'gotta', 'per', 'ofc', 'pls', 'ist', 'wasnt', 'soooo', 'imo', 'non', 'till', 'alt', 'may', 'two', 'im', 'ag', 'oh', 'af', 'xd', 'ye', 'ur', 'ya', 'de', 'pm', 'op', 'th', 'da', 'la', 'bs', 'fp', 'eh', 'yo', 'gb', 'sh', 'co', 'ta', 'ps', 'gw', 'si', 'vs', 'dr', 'es', 'ii', 'tl', 'rn', 'st', 'mu', 'iv', 'em', 'bc', 'ig', 'ms', 'nd', 'un', 'ty', 'rd', 'cc', 'ah', 'se', 'gr', 'iz', 'lo', 'fr', 'hi', 'um', 'dc', 'ad', 'ck', 'fk', 'le', 'gj', 'nt', 'nw', 'en', 'el', 'ex', 'mm', 'tv', 'gf', 'yr', 'va', 'iq', 'bi', 'ea', 'fu', 'ne', 'uh', 'er', 'gs', 'pp', 'mi', 'ja', 'il', 'ol', 'ed', 'os', 'sa', 'je', 'vo', 'ti', 'gl', 'bf', 'aa', 'ch', 'te', 'jo', 'cd', 'al', 'fo', 'tf', 'gm', 'wp', 'hf', 'ue', 'jp', 'dx', 'ss', 'za', 'gd', 'ts', 'og', 'jk', 'ar', 'ux', 'zu', 'fx', 'ko', 'ha', 'eq', 'ga', 'ce', 'yt', 'ft', 'ou', 'rl', 'cs', 'io', 'ww', 'ba', 'mo', 'ir', 'nz', 'rs', 'vv', 'di', 'ip', 'eg', 'mf', 'rp', 'ie', 'ni', 'dm', 'au', 'et', 'qs', 'wa', 'rt', 'ot', 'uk', 'pr', 'dd', 'pe', 'bu', 'ro', 'od', 'tt', 'pt', 'vi', 'hd', 'pi', 'ab', 'mr', 'ca', 'qq', 'tu', 'ay', 'aw', 'hs', 'yh', 'az', 'sg', 'bo', 'ei', 'nu', 'kb', 'pw', 'xx', 'nr', 'oo', 'cm', 'bb', 'ka', 'mh', 'ra', 'nc', 'tp', 'po', 'cu', 'ng', 'fi', 'ho', 'ik', 'tr', 'vr', 'gt', 'mp', 'sd', 'hj', 'gu', 'wn', 'fn', 'qa', 'du', 'ml', 'xi', 'ak', 'mn', 'gz', 'kh', 'mb', 'qu', 'hz', 'kk', 'jj', 'ax', 'ac', 'vb', 'rx', 'bd', 'ey', 'br', 'ee', 'nm', 'yu', 'ri', 'hw', 'yi', 'gi', 'ew', 'xe', 'jh', 'df', 'db', 'ec', 'dn', 'dg', 'ku', 'gp', 'ut', 'ki', 'cg', 'ze', 'vm', 'pa', 'ia', 'tm', 'xt', 'vu', 'ci', 'sk', 'pg', 'oi', 'ke', 'ao', 'cl', 'aj', 'fg', 'ow', 'aq', 'py', 'pd', 'ud', 'fa', 'wh', 'fc', 'kt', 'su', 'cr', 'np', 'mt', 'oc', 'ge', 'cn', 'om', 'nn', 'ju', 'ov', 'pl', 'vc', 'sy', 'wo', 'bg', 'tw', 'wc', 'gy', 'vn', 'sp', 'kg', 'dy', 'cb', 'rc', 'hm', 'ul', 'wm', 'li', 'js', 'ek', 'mq', 'rf', 'uo', 'hc', 'ny', 'lb', 'lt', 'tx', 'cz', 'fs', 'ns', 'iy', 'ks', 'ry', 'tb', 'sc', 'zg', 'oz', 'tq', 'mk', 'bl', 'ap', 'hl', 'cp', 'ld', 'mc', 'pz', 'dw', 'ob', 'ug', 'hq', 'sl', 'rb', 'fl', 'dj', 'ev', 'nk', 'zd', 'zn', 'sb', 'sz', 'ib', 'dp', 'fv', 'px', 'zl', 'fh', 'nl', 'lm', 'ep', 'vy', 'dt', 'oy', 'qt', 'jf', 'wx', 'xo', 'bj', 'nf', 'yn', 'cy', 'vk', 'pf', 'tk', 'nx', 'ym', 'fy', 'fm', 'wk', 'fw', 'jr', 'rz', 'gv', 'pb', 'hu', 'mv', 'sw', 'nb', 'qm', 'pq', 'vw', 'ic', 'jt', 'lr', 'sr', 'bt', 'ln', 'cv', 'nh', 'zx', 'gk', 'pk', 'sm', 'kl', 'xy', 'gh', 'kd', 'bk', 'rk', 'uc', 'sj', 'tc', 'ds', 'ji', 'zm', 'vz', 'lj', 'pu', 'mw', 'km', 'yd', 'pv', 'yc', 'wr', 'ae', 'wg', 'lc', 'qn', 'zi', 'kn', 'ih', 'ys', 'xj', 'xm', 'yq', 'fb', 'xq', 'vx', 'xa', 'xb', 'xc', 'ct', 'rm', 'ls', 'ef', 'rq', 'fd', 'uw', 'gn', 'nj', 'many', 'one', 'would', 'even', 'could', 'another', 'iii', 'ten', 'tho', 'doesnt', 'ffxivs', 'ffxiv', 'donts', 'dont', 'bdos', 'bdo', 'ffs', 'ff', 'las', 'isnt', 'theres', 'des', 'ccs', 'cos', 'rds', 'los', 'bos', 'oce', 'https', 'mis', 'nes', '많은', '보는', '좀더', '유저', 'رو', '드뎌', '디아', '그건', '공짜', '돈은', 'há', 'dl', 'mı', 'aí', 'şu', 'sv', 'lá', 'wi', '거라', '된다', '인기', '다른', '처럼', '골드', '이미', '믿고', '말들', '또한', '앤드', '하기', '위한', '해당', '없다', '하면', '되는', '아주', '있는', '많다', '물론', '그저', '이라', '뭐라', '하며', '굳이', '관한', '끌며', '하는', '맛이', '한글', '하지', '조금', '어느', '나서', '졸라', '재미', '있다', 'dá', 'ノ彡', 'ça', 'už', 'aaaaaaalllllttttsssss', 'game', 'yes', 'really', 'lot', 'much', 'thing', 'still', 'also', 'cant', 'ok', 'actually', 'cannot', 'pog', 'literally', 'alot', 'ive', 'however', 'etc', 'thats', 'ass', 'far', 'everything', 'every', 'since', 'something', 'ever', 'never', 'nan', 'seem', 'ton', 'though', 'overall', 'yet', 'able', 'due', 'sure', 'without', 'highly', 'anything', 'part', 'might', 'kinda', 'probably', 'maybe', 'please', 'lol', 'alway', 'booba', 'honestly', 'almost', 'yeah', 'instead', 'bite', 'definitely', 'absolutely', 'finally', 'early', 'extremely', 'especially', 'basically', 'daily', 'simply', 'completely', 'currently', 'mostly', 'easily', 'friendly', 'incredibly', 'totally', 'quickly', 'personally', 'usually', 'seriously', 'truly', 'unfortunately', 'hopefully', 'constantly', 'sadly', 'barely', 'exactly', 'weekly', 'fairly', 'nearly', 'likely', 'insanely', 'visually', 'eventually', 'slowly', 'fully', 'casually', 'lovely', 'holy', 'clearly', 'heavily', 'normally', 'family', 'apparently', 'properly', 'poorly', 'obviously', 'certainly', 'surprisingly', 'slightly', 'immediately', 'instantly', 'genuinely', 'overly', 'overwhelmingly', 'essentially', 'entirely', 'perfectly', 'generally', 'mainly', 'recently', 'rly', 'ultimately', 'randomly', 'thoroughly', 'fly', 'possibly', 'mindlessly', 'realy', 'monthly', 'amazingly', 'initially', 'strongly', 'badly', 'smoothly', 'silly', 'particularly', 'rarely', 'nicely', 'roughly', 'hardly', 'beautifully', 'ugly', 'terribly', 'relatively', 'utterly', 'painfully', 'apply', 'massively', 'rely', 'greatly', 'ridiculously', 'graphically', 'seemingly', 'lately', 'directly', 'practically', 'specially', 'actively', 'suddenly', 'typically', 'remotely', 'correctly', 'purely', 'surely', 'decently', 'technically', 'frankly', 'originally', 'repeatedly', 'supposedly', 'importantly', 'thankfully', 'occasionally', 'luckily', 'specifically', 'significantly', 'potentially', 'consistently', 'horribly', 'numbingly', 'definetly', 'frequently', 'absolutly', 'mechanically', 'differently', 'automatically', 'safely', 'definitly', 'stupidly', 'lastly', 'happily', 'extremly', 'gladly', 'endlessly', 'pleasantly', 'largely', 'solely', 'basicly', 'effectively', 'fortunately', 'negatively', 'severely', 'shortly', 'necessarily', 'completly', 'prolly', 'infinitely', 'permanently', 'virtually', 'finaly', 'freely', 'unbelievably', 'additionally', 'actualy', 'previously', 'mentally', 'unnecessarily', 'aesthetically', 'manually', 'reply', 'godly', 'reasonably', 'immensely', 'needlessly', 'intentionally', 'officially', 'exclusively', 'objectively', 'desperately', 'successfully', 'accidentally', 'accordingly', 'primarily', 'legitimately', 'vastly', 'definately', 'temporarily', 'unlikely', 'firstly', 'carefully', 'exceptionally', 'laughably', 'partially', 'oddly', 'supply', 'rlly', 'equally', 'blatantly', 'admittedly', 'truely', 'progressively', 'approximately', 'rapidly', 'continuously', 'hugely', 'deadly', 'deeply', 'wildly', 'regularly', 'totaly', 'bully', 'efficiently', 'drastically', 'secondly', 'arguably', 'excessively', 'patiently', 'simultaneously', 'gradually', 'realistically', 'mildly', 'naturally', 'increasingly', 'physically', 'newly', 'litterally', 'srsly', 'weirdly', 'artificially', 'continually', 'absurdly', 'unfriendly', 'moly', 'strangely', 'definetely', 'hilariously', 'miserably', 'similarly', 'steadily', 'wisely', 'scantily', 'dangerously', 'confidently', 'sincerely', 'positively', 'aggressively', 'critically', 'purposely', 'lonely', 'falsely', 'royally', 'unexpectedly', 'flawlessly', 'mindnumbingly', 'promptly', 'stunningly', 'wonderfully', 'purposefully', 'hopelessly', 'comically', 'merely', 'horrendously', 'strictly', 'comfortably', 'precisely', 'timely', 'suprisingly', 'ungodly', 'litteraly', 'awfully', 'ply', 'defiantly', 'definitively', 'pally', 'undoubtedly', 'horrifically', 'unfairly', 'unironically', 'inevitably', 'dreadfully', 'eagerly', 'deliberately', 'wholeheartedly', 'fundamentally', 'widely', 'strategically', 'lively', 'gameply', 'competitively', 'allegedly', 'lazily', 'seamlessly', 'closely', 'ironically', 'conveniently', 'accurately', 'briefly', 'moderately', 'separately', 'tremendously', 'basicaly', 'namely', 'understandably', 'boringly', 'multiply', 'reliably', 'presumably', 'willingly', 'personaly', 'cleverly', 'politically', 'higly', 'emotionally', 'overtly', 'unbearably', 'partly', 'socially', 'blindly', 'indefinitely', 'brilliantly', 'relly', 'effortlessly', 'magically', 'considerably', 'finnaly', 'respectfully', 'refreshingly', 'disgustingly', 'awesomely', 'wrongfully', 'ally', 'abysmally', 'spectacularly', 'substantially', 'offensively', 'lightly', 'imply', 'impossibly', 'costly', 'willy', 'aimlessly', 'cautiously', 'unfortunatly', 'financially', 'trully', 'cheaply', 'obnoxiously', 'exponentially', 'shockingly', 'unholy', 'rightfully', 'jolly', 'excruciatingly', 'fantastically', 'literaly', 'sparkly', 'handedly', 'addictively', 'grossly', 'readily', 'dramatically', 'openly', 'preferably', 'iguwcwcpqly', 'technologically', 'actully', 'holly', 'pointlessly', 'astonishingly', 'extraordinarily', 'firmly', 'individually', 'sorely', 'extensively', 'especialy', 'propably', 'impressively', 'arbitrarily', 'evenly', 'unjustly', 'intensely', 'sexually', 'majorly', 'marginally', 'annoyingly', 'vaguely', 'normaly', 'eventualy', 'culturally', 'smelly', 'frustratingly', 'kindly', 'belly', 'comparatively', 'accidently', 'procedurally', 'suspiciously', 'passively', 'notably', 'lutely', 'paly', 'artifically', 'glaringly', 'brainlessly', 'exceedingly', 'hourly', 'eternally', 'loudly', 'manly', 'astronomically', 'presently', 'deffinately', 'unreasonably', 'religiously', 'forcefully', 'woefully', 'singlehandedly', 'explicitly', 'bigly', 'harshly', 'theoretically', 'monetarily', 'easly', 'obsessively', 'prohibitively', 'wrongly', 'inherently', 'insanly', 'ideally', 'warmly', 'visualy', 'fiddly', 'thematically', 'overally', 'defenetly', 'probly', 'extreamly', 'hopfully', 'realllly', 'wholly', 'minimally', 'miraculously', 'monopoly', 'effectivly', 'plainly', 'egregiously', 'montly', 'hysterically', 'loosely', 'definatly', 'acutally', 'alternatively', 'subjectively', 'astoundingly', 'pitifully', 'optimally', 'concurrently', 'measly', 'swiftly', 'eloquently', 'helplessly', 'artistically', 'quietly', 'commonly', 'humanly', 'defenitly', 'incrementally', 'brutally', 'thinly', 'gently', 'functionally', 'immediatly', 'basiclly', 'frendly', 'truthfully', 'awkwardly', 'mathematically', 'orderly', 'jelly', 'nightly', 'yearly', 'instantaneously', 'notoriously', 'abundantly', 'uncomfortably', 'obiously', 'worldly', 'absoulutely', 'uniquely', 'dayly', 'automaticly', 'consequently', 'perpetually', 'insufferably', 'excellently', 'tightly', 'consecutively', 'psychologically', 'traditionally', 'daly', 'luckly', 'unabashedly', 'tediously', 'lovingly', 'inconsistently', 'sparingly', 'collectively', 'heavly', 'thirdly', 'freakishly', 'sufficiently', 'globally', 'muscly', 'bascially', 'constanly', 'linearly', 'masterfully', 'ly', 'trivially', 'broadly', 'poly', 'vertically', 'remarkably', 'fluidly', 'espically', 'meticulously', 'depressingly', 'embarrassingly', 'questionably', 'doubly', 'literately', 'unfortunatelly', 'relentlessly', 'appropriately', 'horizontally', 'partialy', 'logically', 'diligently', 'periodically', 'thoughtfully', 'evidently', 'bogglingly', 'otherworldly', 'remotly', 'digitally', 'aply', 'shamelessly', 'indirectly', 'statistically', 'noticeably', 'phenomenally', 'unrealistically', 'litterly', 'actuallly', 'predictably', 'absolutelly', 'sneakily', 'acually', 'regrettably', 'heartily', 'rediculously', 'interestingly', 'specialy', 'thouroughly', 'boldly', 'reluctantly', 'numbly', 'silently', 'scantly', 'insultingly', 'leisurely', 'repetitively', 'secretly', 'increadibly', 'intuitively', 'violently', 'obscenely', 'publicly', 'rationally', 'butterfly', 'anxiously', 'adequately', 'bluntly', 'subtly', 'cosmetically', 'independently', 'surly', 'aptly', 'meaningfully', 'abruptly', 'deffinitely', 'conceptually', 'idly', 'barelly', 'responsibly', 'delicately', 'supremely', 'unapologetically', 'hesitantly', 'atrociously', 'lowly', 'generelly', 'assuredly', 'unacceptably', 'fluently', 'intellectually', 'activly', 'gloriously', 'audibly', 'mindbogglingly', 'fondly', 'admitedly', 'confusingly', 'unimaginably', 'dearly', 'profoundly', 'honesly', 'narrowly', 'appallingly', 'spontaneously', 'respectively', 'awefully', 'sparsely', 'improperly', 'radically', 'racially', 'verbally', 'usally', 'internally', 'tirelessly', 'impulsively', 'proudly', 'incessantly', 'monstrously', 'bubbly', 'agonisingly', 'innocently', 'subconsciously', 'bewilderingly', 'illegally', 'burly', 'vividly', 'modestly', 'girly', 'jarringly', 'invariably', 'visibly', 'knowingly', 'scientifically', 'reportedly', 'routinely', 'forcibly', 'severly', 'rabidly', 'momentarily', 'tactically', 'sally', 'literely', 'cartoonishly', 'abhorrently', 'expressly', 'locally', 'sharply', 'mysteriously', 'heroically', 'gleefully', 'constatly', 'sporadically', 'prematurely', 'justifiably', 'generously', 'subsequently', 'superficially', 'neatly', 'greedily', 'mashingly', 'smartly', 'elderly', 'favorably', 'furiously', 'unambiguously', 'passionately', 'ludicrously', 'mistakenly', 'marginaly', 'july', 'expertly', 'jokingly', 'squarely', 'unsurprisingly', 'externally', 'freshly', 'painstakingly', 'definitelly', 'generically', 'haphazardly', 'tragically', 'undeniably', 'finely', 'unusually', 'naively', 'fiercely', 'ethically', 'stylistically', 'massivly', 'absoloutly', 'generaly', 'occasionaly', 'extreemly', 'bazly', 'halfingly', 'organically', 'litereally', 'essencially', 'competently', 'porly', 'intermediately', 'extremaly', 'understandingly', 'liberally', 'dishearteningly', 'bizarrely', 'pratically', 'faintly', 'unforunately', 'oily', 'arguebly', 'outrageously', 'occassionally', 'deliciously', 'morally', 'autoreply', 'chiefly', 'dankly', 'dfinitely', 'stongly', 'soullessly', 'sustainably', 'intentionaly', 'dialy', 'defenitely', 'sequentially', 'fatally', 'cynically', 'unequivocally', 'comnpletely', 'exrteeemely', 'noticably', 'lubly', 'tecnically', 'brightly', 'biweekly', 'pathetically', 'skilly', 'delightfully', 'gnarly', 'friedly', 'comparably', 'littrally', 'charmingly', 'internationally', 'curently', 'definatitly', 'ndly', 'absouletly', 'epicly', 'stochastlicaly', 'unassumingly', 'erroneously', 'sully', 'senselessly', 'incorrectly', 'popably', 'proposterously', 'pooly', 'phsycologically', 'hollowly', 'regretfully', 'mostely', 'idiotically', 'entirelly', 'arrogantly', 'unfortunetly', 'calmly', 'espcially', 'seemly', 'slighly', 'resupply', 'unsuccessfully', 'unioronically', 'rollingly', 'blankly', 'avidly', 'tangibly', 'predatorily', 'entertainingly', 'blessedly', 'joyfully', 'distinctively', 'drammatically', 'alphabetically', 'selectively', 'extreemely', 'digustingly', 'beastly', 'legitmately', 'hopefiully', 'strickly', 'breakingly', 'rarly', 'pesonally', 'zodly', 'unrightfully', 'innately', 'wyeliminowaly', 'deffinetely', 'ackchyually', 'tottally', 'fraudly', 'impotently', 'esepcially', 'catastrophically', 'nominally', 'easely', 'urgently', 'technikly', 'magicaly', 'technicaly', 'absolultely', 'hypothetically', 'simpily', 'mystically', 'additonally', 'gorgeously', 'indescribably', 'shockigly', 'deffinetly', 'litreally', 'starkly', 'expecially', 'structurely', 'ocassionally', 'soundly', 'unhealthily', 'apparentely', 'exxxxtreamly', 'preemptively', 'vocally', 'untimely', 'patronizingly', 'deflatingly', 'cheerily', 'conservatively', 'classically', 'senserly', 'forcedly', 'proprely', 'literalaly', 'heinously', 'muchly', 'obvisioully', 'reallllly', 'unashamedly', 'gracefully', 'distinctly', 'continualy', 'butterflysly', 'rilly', 'tastefully', 'immeasurably', 'underbelly', 'flowly', 'gratuitiously', 'goodly', 'indiscrimantely', 'crucially', 'relaly', 'incidentally', 'physiologically', 'alternatiely', 'tolerably', 'genuinly', 'unfortunally', 'plesantly', 'immedianetly', 'crearly', 'drainingly', 'seriosly', 'compleatly', 'espeically', 'mindblowingly', 'figuratively', 'precisly', 'categorically', 'espeshailly', 'terrifically', 'espacially', 'immeditely', 'uniformly', 'litearlly', 'unnaturaly', 'initialy', 'dutifully', 'virutally', 'longely', 'immensly', 'genially', 'immidiately', 'eerily', 'heavely', 'irritatingly', 'convienently', 'unethically', 'entierly', 'genuininly', 'magicly', 'ily', 'disrespectfully', 'abyssmally', 'disturbingly', 'infinitly', 'absoluitely', 'thusly', 'womanly', 'reeeeealy', 'deffinitly', 'fearfully', 'beautifuly', 'dangerouly', 'enfuriantingly', 'curiously', 'vigorously', 'halfly', 'synergetically', 'weekely', 'agressively', 'secoundly', 'mindnumblingly', 'universally', 'inefficiently', 'beuatifly', 'exactely', 'inexplicably', 'inexcusably', 'outstandingly', 'dominantly', 'unwillingly', 'braindeadly', 'stably', 'quckly', 'realllllly', 'exactlly', 'anomaly', 'enjoyably', 'retroactively', 'manically', 'twinkly', 'probrobly', 'unmistakably', 'humanely', 'unavoidably', 'monumently', 'siply', 'obivously', 'rally', 'eyewateringly', 'controversially', 'abusively', 'komoly', 'hely', 'frantically', 'reaaaaally', 'asianly', 'regularily', 'contextually', 'conceivably', 'interminably', 'simoultaniously', 'dazzlingly', 'inactively', 'completily', 'wrinkly', 'barerly', 'seriusly', 'miserabily', 'extremtly', 'particularily', 'incredebly', 'sincerally', 'demographically', 'duly', 'imminently', 'actuly', 'jaringly', 'hourlly', 'addictingly', 'formulatically', 'numingly', 'slowely', 'midly', 'ominously', 'mostally', 'dumbly', 'negligibly', 'incompitently', 'causually', 'immidately', 'reaaaally', 'sudently', 'softly', 'slithly', 'feasably', 'confidentally', 'predominately', 'approxmatietly', 'unexcitingly', 'carelessly', 'frighteningly', 'passably', 'throughoutly', 'barly', 'bearly', 'intermittently', 'unintentionally', 'snuggly', 'richly', 'brolly', 'agduly', 'expectionally', 'strikingly', 'litrilaly', 'probbably', 'overwelmingly', 'diffrently', 'increibly', 'sickeningly', 'commanly', 'immediantly', 'apperantly', 'mistakely', 'authentically', 'morbidly', 'additionaly', 'espesially', 'defiently', 'begrudgingly', 'panoply', 'plently', 'hoooly', 'epicaly', 'jiggly', 'insidiously', 'coincidently', 'thanksfully', 'specificly', 'littearlly', 'exremely', 'stubbornly', 'easyly', 'schmoly', 'tenderly', 'princely', 'nsanely', 'irrevocably', 'homeopathically', 'stiflingly', 'curdlingly', 'inducingly', 'joylessly', 'summarily', 'densely', 'accualy', 'distractingly', 'compeltely', 'meaninglessly', 'dynamically', 'imediatly', 'accully', 'optically', 'molly', 'creatively', 'accedently', 'jerkily', 'fundementally', 'unfortunetaly', 'realisticly', 'harmoniously', 'hly', 'beautfiully', 'defaultly', 'unfortunettly', 'unfortunely', 'activily', 'annually', 'persistently', 'incrediably', 'sluggishly', 'visially', 'bitterly', 'olnly', 'intensively', 'regionally', 'absysmally', 'beutifully', 'definatelly', 'rightly', 'prohibitavely', 'genrelly', 'aproximately', 'cordialy', 'characteristically', 'scarcely', 'immedietly', 'fundamentaly', 'inherintly', 'unfrotunately', 'excitingly', 'genuenly', 'artisticly', 'eyecatchily', 'actaully', 'usualy', 'intantly', 'ordinarily', 'historically', 'apperently', 'trruly', 'virtualy', 'steadfastly', 'magnificently', 'stiffly', 'inversely', 'uncommonly', 'unbelieveably', 'detaily', 'tely', 'weakly', 'rarelly', 'likelly', 'ridulously', 'comply', 'narratively', 'manualy', 'peacefully', 'luckely', 'inhumanly', 'actutally', 'equaly', 'perspicaciously', 'complicatedly', 'dryly', 'subsenquently', 'clumsily', 'extreeeeeeeemly', 'amazingingly', 'definietly', 'strategicly', 'essentialy', 'convieniently', 'blowingly', 'especally', 'impeccably', 'deffinatly', 'specificallly', 'charitably', 'actlly', 'chilly', 'splendidly', 'casualy', 'relevantly', 'optionally', 'unceremoniously', 'competetively', 'immutably', 'mockingly', 'truthfuly', 'accually', 'factually', 'inanely', 'punishingly', 'unofrtunately', 'unrealisticly', 'luckilly', 'especailly', 'frenkly', 'possitively', 'definetelly', 'siriusly', 'graphicly', 'approximetly', 'swimmingly', 'throughly', 'paywally', 'seemlessly', 'surreptitiously', 'lastely', 'affectionately', 'strainingly', 'ingloriously', 'identically', 'obvisly', 'flashly', 'thoroghly', 'expectedly', 'bafflingly', 'willfully', 'concurently', 'basicilly', 'genetically', 'coarsely', 'helpfully', 'cozingly', 'retardedly', 'medly', 'relativly', 'deathly', 'absically', 'intrusively', 'ruffly', 'acctually', 'repetatively', 'handily', 'definently', 'annualy', 'unecessarily', 'esentially', 'continuosly', 'overwhemly', 'professionally', 'numblingly', 'extrmely', 'correspondingly', 'unfortunaly', 'diagonally', 'exorbitantly', 'inmediatly', 'compleately', 'dismally', 'economically', 'earlly', 'defintely', 'efficently', 'immately', 'unwittingly', 'primarely', 'revoltingly', 'heavenly', 'obviusly', 'defnitly', 'staunchly', 'ungainly', 'thoughtlessly', 'duscustingly', 'nelly', 'aditionally', 'reallly', 'supriseingly', 'porbably', 'extreeeemeely', 'propperly', 'assembly', 'undefinetly', 'isliterally', 'reeeeeeeeeeeeely', 'narly', 'exstatically', 'accidentily', 'slickly', 'horridly', 'purposfly', 'politely', 'imposibly', 'halfheartedly', 'tingly', 'sincirely', 'overwhelmly', 'defently', 'synchronously', 'mily', 'mercifully', 'uncontrollably', 'finallly', 'autoamtically', 'cinstantly', 'staly', 'offenively', 'legally', 'fourthly', 'annyoingly', 'godawfully', 'barealy', 'crushingly', 'whatifitypedlikethiscanyoureaditnicely', 'defintly', 'emmensely', 'definetily', 'devastatingly', 'causally', 'natively', 'overwhlemingly', 'deservedly', 'failrly', 'intelligently', 'imidiatly', 'greedly', 'tuely', 'aparently', 'geniuenly', 'temporarly', 'ostensibly', 'alternately', 'suitably', 'gratuitously', 'enthusiastically', 'suddently', 'faithfully', 'sinfully', 'undeservedly', 'monumentally' ]

synonyms ={'pc':'computer', 'xp':'experience', 'combat':'fight', 'servers' : 'server', 'things' : 'thing', 'items' : 'item', 'goods' : 'good',  'mmos' : 'mmo', 'storys' : 'story', 'greats' : 'great', 'moneys' : 'money',  'bads' : 'bad', 'ways' : 'way', 'diablos' : 'diablo', 'arks' : 'ark', 'news' : 'new',  'amazons' : 'amazon', 'worlds' : 'world', 'gameplays' : 'gameplay', 'mobiles' : 'mobile',  'days' : 'day', 'firsts' : 'first', 'everythings' : 'everything', 'systems' : 'system',  'worths' : 'worth', 'players' : 'player', 'koreans' : 'korean', 'nothings' : 'nothing',  'somethings' : 'something', 'progressions' : 'progression', 'differents' : 'different',  'arounds' : 'around', 'golds' : 'gold', 'mains' : 'main', 'mmorpgs' : 'mmorpg',  'tiers' : 'tier', 'everyones' : 'everyone', 'arpgs' : 'arpg', 'hards' : 'hard', 'skillss' : 'skills',  'rngs' : 'rng', 'dailys' : 'daily', 'skills' : 'skill', 'dungeons' : 'dungeon', 'currencys' : 'currency',  'lvls' : 'lvl', 'anyones' : 'anyone', 'elses' : 'else', 'kinds' : 'kind', 'problems' : 'problem',  'aspects' : 'aspect', 'conveniences' : 'convenience', 'versions' : 'version', 'someones' : 'someone',  'highs' : 'high', 'alots' : 'alot', 'genders' : 'gender', 'hours' : 'hour', 'attentions' : 'attention',  'weeks' : 'week', 'opinions' : 'opinion', 'facts' : 'fact', 'npcs' : 'npc', 'guilds' : 'guild',  'olds' : 'old', 'ilvls' : 'ilvl', 'casuals' : 'casual', 'islands' : 'island', 'pcs' : 'pc',  'paths' : 'path', 'rpgs' : 'rpg', 'tons' : 'ton', 'pers' : 'per', 'areas' : 'area',  'positives' : 'positive', 'hells' : 'hell', 'months' : 'month', 'regions' : 'region',  'simulators' : 'simulator', 'paywalls' : 'paywall', 'auras' : 'aura', 'ques' : 'que',  'strongholds' : 'stronghold', 'alts' : 'alt', 'futures' : 'future', 'genres' : 'genre',  'moments' : 'moment', 'matchmakings' : 'matchmaking', 'smilegates' : 'smilegate',  'guardians' : 'guardian', 'logins' : 'login', 'options' : 'option', 'dialogues' : 'dialogue',  'subscriptions' : 'subscription', 'persons' : 'person', 'asians' : 'asian', 'females' : 'female',  'abilitys' : 'ability', 'ideas' : 'idea', 'extras' : 'extra', 'customizations' : 'customization',  'randoms' : 'random', 'examples' : 'example', 'wallets' : 'wallet', 'whatevers' : 'whatever',  'publishers' : 'publisher', 'weeklys' : 'weekly', 'specials' : 'special', 'larges' : 'large',  'years' : 'year', 'uis' : 'ui', 'anyways' : 'anyway', 'cameras' : 'camera', 'musics' : 'music',  'storylines' : 'storyline', 'depths' : 'depth', 'ffxivs' : 'ffxiv', 'poors' : 'poor',  'maintenances' : 'maintenance', 'efforts' : 'effort', 'basics' : 'basic', 'gods' : 'god',  'asides' : 'aside', 'epics' : 'epic', 'monetizations' : 'monetization', 'mins' : 'min',  'videos' : 'video', 'windows' : 'window', 'arenas' : 'arena', 'errors' : 'error', 'modes' : 'mode',  'monsters' : 'monster', 'movements' : 'movement', 'todays' : 'today',  'tutorials' : 'tutorial', 'premiums' : 'premium', 'animes' : 'anime', 'straights' : 'straight',  'grindfests' : 'grindfest', 'choices' : 'choice', 'explorations' : 'exploration', 'friends' : 'friend',  'mechanics' : 'mechanic', 'arts' : 'art', 'cosmetics' : 'cosmetic', 'warcrafts' : 'warcraft',  'youtubes' : 'youtube', 'males' : 'male', 'creations' : 'creation', 'differences' : 'difference',  'chores' : 'chore', 'fests' : 'fest', 'hunters' : 'hunter', 'standards' : 'standard', 'abyssals' : 'abyssal',  'mokokos' : 'mokoko', 'autos' : 'auto', 'specifics' : 'specific', 'millions' : 'million',  'micros' : 'micro', 'playtimes' : 'playtime', 'enemys' : 'enemy', 'controllers' : 'controller',  'downsides' : 'downside', 'stats' : 'stat', 'tls' : 'tl', 'gamers' : 'gamer', 'ggs' : 'gg',  'situations' : 'situation', 'lores' : 'lore', 'souls' : 'soul', 'favorites' : 'favorite',  'optionals' : 'optional', 'nights' : 'night', 'russians' : 'russian', 'dollars' : 'dollar',  'ppls' : 'ppl', 'coms' : 'com', 'swords' : 'sword', 'animations' : 'cutscen', 'texts' : 'text',  'minutes' : 'minute', 'keyboards' : 'keyboard', 'visuals' : 'visual', 'continents' : 'continent',  'definitions' : 'definition', 'materials' : 'material', 'gearscores' : 'gearscore', 'classics' : 'classic',  'mornings' : 'morning', 'cents' : 'cent', 'cinematics' : 'cinematic', 'rapports' : 'rapport',  'periods' : 'period', 'bots' : 'bot', 'crystals' : 'crystal', 'starters' : 'starter', 'decisions' : 'decision',  'menus' : 'menu', 'pros' : 'pro', 'events' : 'event', 'customers' : 'customer', 'audiences' : 'audience',  'ops' : 'op', 'grinders' : 'grinder', 'perspectives' : 'perspective', 'betas' : 'beta', 'hundreds' : 'hundred',  'playerbases' : 'playerbase', 'engines' : 'engine', 'winners' : 'winner', 'investments' : 'investment',  'dialogs' : 'dialog', 'conclusions' : 'conclusion', 'connections' : 'connection', 'developers' : 'developer',  'products' : 'product', 'aoes' : 'aoe', 'translations' : 'translation', 'environments' : 'environment',  'computers' : 'computer', 'languages' : 'language', 'hots' : 'hot', 'devs' : 'dev', 'rares' : 'rare',  'solutions' : 'solution', 'giants' : 'giant', 'cooldowns' : 'cooldown', 'warframes' : 'warframe',  'directions' : 'direction', 'deaths' : 'death', 'completionists' : 'completionist', 'goals' : 'goal',  'editions' : 'edition', 'extremes' : 'extreme', 'developments' : 'development', 'weapons' : 'weapon',  'breaths' : 'breath', 'complaints' : 'complaint', 'masterpieces' : 'masterpiece', 'advices' : 'advice',  'paladins' : 'paladin', 'europeans' : 'european', 'roads' : 'road', 'mages' : 'mage', 'users' : 'user',  'kmmos' : 'kmmo', 'alphas' : 'alpha', 'commons' : 'common', 'uninstalls' : 'uninstall', 'scenes' : 'scene',  'dlcs' : 'dlc', 'warriors' : 'warrior', 'concepts' : 'concept', 'locations' : 'location', 'resources' : 'resource',  'achievements' : 'achievement', 'prologues' : 'prologue', 'ilevels' : 'ilevel', 'failures' : 'failure',  'streamers' : 'streamer', 'populations' : 'population', 'secrets' : 'secret', 'minimums' : 'minimum',  'playstyles' : 'playstyle', 'payments' : 'payment', 'cutscenes' : 'cutscene', 'tabs' : 'tab',  'performances' : 'performance', 'mmoarpgs' : 'mmoarpg', 'boys' : 'boy', 'berserkers' : 'berserker',  'cliches' : 'cliche', 'studios' : 'studio', 'completions' : 'completion', 'sieges' : 'siege',  'interfaces' : 'interface', 'frustrations' : 'frustration', 'elements' : 'element', 'blizzards' : 'blizzard',  'localizations' : 'localization', 'recommendations' : 'recommendation', 'collectibles' : 'collectible',  'disasters' : 'disaster', 'hearts' : 'heart', 'gvgs' : 'gvg', 'gems' : 'gem', 'missions' : 'mission',  'comparisons' : 'comparison', 'metas' : 'meta', 'evils' : 'evil', 'casinos' : 'casino', 'setups' : 'setup',  'expansions' : 'expansion', 'patients' : 'patient', 'heros' : 'hero', 'defs' : 'def', 'seas' : 'sea',  'feedbacks' : 'feedback', 'graphics' : 'graphic', 'towns' : 'town', 'assassins' : 'assassin',  'collections' : 'collection', 'vibes' : 'vibe', 'thousands' : 'thousand', 'greeds' : 'greed',  'selections' : 'selection', 'roles' : 'role', 'mobas' : 'moba', 'killers' : 'killer',  'customisations' : 'customisation', 'questlines' : 'questline', 'downtimes' : 'downtime',  'wifes' : 'wife', 'decades' : 'decade', 'cookies' : 'cookie', 'incentives' : 'incentive',  'arguments' : 'argument', 'tokens' : 'token', 'combos' : 'combo', 'fillers' : 'filler',  'enhancements' : 'enhancement', 'dragons' : 'dragon', 'msqs' : 'msq', 'transactions' : 'transaction',  'requirements' : 'requirement', 'childrens' : 'children', 'toons' : 'toon', 'wasters' : 'waster',  'demons' : 'demon', 'interactions' : 'interaction', 'gunslingers' : 'gunslinger', 'americas' : 'america',  'maris' : 'mari', 'odds' : 'odd', 'blades' : 'blade', 'timers' : 'timer', 'communications' : 'communication',  'waifus' : 'waifu', 'everybodys' : 'everybody', 'oceans' : 'ocean', 'tomes' : 'tome', 'hairs' : 'hair',  'hybrids' : 'hybrid', 'additions' : 'addition', 'sakes' : 'sake', 'addictions' : 'addiction', 'hrs' : 'hr',  'degrees' : 'degree', 'reputations' : 'reputation', 'combinations' : 'combination', 'potions' : 'potion',  'dudes' : 'dude', 'ses' : 'se', 'individuals' : 'individual', 'unas' : 'una', 'responses' : 'response',  'narratives' : 'narrative', 'dynamics' : 'dynamic', 'rotations' : 'rotation', 'bcs' : 'bc', 'girls' : 'girl',  'fives' : 'five', 'kings' : 'king', 'appearances' : 'appearance', 'substances' : 'substance',  'microtransactions' : 'microtransaction', 'artists' : 'artist', 'rampants' : 'rampant',  'percents' : 'percent', 'songs' : 'song', 'disappointments' : 'disappointment', 'humans' : 'human',  'dozens' : 'dozen', 'manners' : 'manner', 'impressions' : 'impression', 'weebs' : 'weeb', 'chests' : 'chest',  'childs' : 'child', 'percentages' : 'percentage', 'boobas' : 'booba', 'legions' : 'legion', 'realms' : 'realm',  'forums' : 'forum', 'layouts' : 'layout', 'neutrals' : 'neutral', 'formulas' : 'formula', 'creators' : 'creator',  'sims' : 'sim', 'objectives' : 'objective', 'soundtracks' : 'soundtrack', 'foods' : 'food', 'aesthetics' : 'aesthetic',  'improvements' : 'improvement', 'intros' : 'intro', 'advancements' : 'advancement', 'ings' : 'ing', 'euros' : 'euro',  'explanations' : 'explanation', 'ssds' : 'ssd', 'breakers' : 'breaker', 'veterans' : 'veteran', 'apps' : 'app',  'atmospheres' : 'atmosphere', 'jrpgs' : 'jrpg', 'healers' : 'healer', 'activitys' : 'activity',  'opponents' : 'opponent', 'topics' : 'topic', 'transmogs' : 'transmog', 'sundays' : 'sunday', 'laptops' : 'laptop',  'alternatives' : 'alternative', 'satisfactions' : 'satisfaction', 'resolutions' : 'resolution',  'statics' : 'static', 'fighters' : 'fighter', 'nightmares' : 'nightmare', 'summoners' : 'summoner',  'methods' : 'method', 'exceptions' : 'exception', 'wilds' : 'wild', 'compensations' : 'compensation',  'seagulls' : 'seagull', 'bros' : 'bro', 'medias' : 'media', 'movies' : 'movie', 'sales' : 'sale',  'das' : 'da', 'consumers' : 'consumer', 'conversations' : 'conversation', 'respawns' : 'respawn',  'deathblades' : 'deathblade', 'afternoons' : 'afternoon', 'strengths' : 'strength', 'tvs' : 'tv',  'pandas' : 'panda', 'ratios' : 'ratio', 'americans' : 'american', 'presentations' : 'presentation',  'scenarios' : 'scenario', 'variations' : 'variation', 'priests' : 'priest', 'playthroughs' : 'playthrough',  'beginners' : 'beginner', 'probs' : 'prob', 'cks' : 'ck', 'besides' : 'beside', 'gunners' : 'gunner',  'preferences' : 'preference', 'artillerists' : 'artillerist', 'introductions' : 'introduction',  'ultimates' : 'ultimate', 'sessions' : 'session', 'strikers' : 'striker', 'planets' : 'planet',  'statements' : 'statement', 'vpns' : 'vpn', 'maths' : 'math', 'passives' : 'passive', 'merchants' : 'merchant',  'implementations' : 'implementation', 'equivalents' : 'equivalent', 'muitos' : 'muito', 'towards' : 'toward',  'bfs' : 'bf', 'bricks' : 'brick', 'relationships' : 'relationship', 'clients' : 'client', 'borderlines' : 'borderline',  'pheons' : 'pheon', 'billions' : 'billion', 'specs' : 'spec', 'sorcs' : 'sorc', 'vas' : 'va',  'commitments' : 'commitment', 'sometimes' : 'sometime', 'newbies' : 'newbie', 'collectors' : 'collector',  'threats' : 'threat', 'dads' : 'dad', 'adventurers' : 'adventurer', 'bikinis' : 'bikini', 'conversions' : 'conversion',  'respecs' : 'respec', 'criticisms' : 'criticism', 'platforms' : 'platform', 'vendors' : 'vendor', 'adults' : 'adult',  'payoffs' : 'payoff', 'annoyances' : 'annoyance', 'egs' : 'eg', 'tripods' : 'tripod', 'idiots' : 'idiot',  'sellers' : 'seller', 'turds' : 'turd', 'consumables' : 'consumable', 'carrots' : 'carrot',  'bullets' : 'bullet', 'capitals' : 'capital', 'foundations' : 'foundation', 'noobs' : 'noob',  'australians' : 'australian', 'meats' : 'meat', 'providers' : 'provider', 'members' : 'member',  'pleasures' : 'pleasure', 'crawlers' : 'crawler', 'keybinds' : 'keybind', 'accomplishments' : 'accomplishment',  'heavens' : 'heaven', 'volumes' : 'volume', 'givers' : 'giver', 'krpgs' : 'krpg', 'timesinks' : 'timesink',  'generations' : 'generation', 'shadowhunters' : 'shadowhunter', 'elites' : 'elite', 'hordes' : 'horde',  'triports' : 'triport', 'purchasables' : 'purchasable', 'poops' : 'poop', 'productions' : 'production',  'throats' : 'throat', 'pvpers' : 'pvper', 'metrics' : 'metric', 'portals' : 'portal', 'textures' : 'texture',  'lolis' : 'loli', 'memes' : 'meme', 'revenues' : 'revenue', 'presets' : 'preset', 'secs' : 'sec', 'icons' : 'icon',  'minimaps' : 'minimap', 'marathons' : 'marathon', 'talents' : 'talent', 'markers' : 'marker', 'hotbars' : 'hotbar',  'trailers' : 'trailer', 'girlfriends' : 'girlfriend', 'websites' : 'website', 'checklists' : 'checklist',  'managers' : 'manager', 'desktops' : 'desktop', 'motivations' : 'motivation', 'superiors' : 'superior',  'notifications' : 'notification', 'clickers' : 'clicker', 'hips' : 'hip', 'exaggerations' : 'exaggeration',  'ers' : 'er', 'barriers' : 'barrier', 'twinks' : 'twink', 'timegates' : 'timegate', 'paychecks' : 'paycheck',  'mountains' : 'mountain', 'ads' : 'ad', 'chickens' : 'chicken', 'expectations' : 'expectation', 'lovers' : 'lover',  'heights' : 'height', 'reactions' : 'reaction', 'minigames' : 'minigame', 'pedos' : 'pedo', 'bois' : 'boi',  'boosters' : 'booster', 'kingdoms' : 'kingdom', 'itemlvls' : 'itemlvl', 'gimmicks' : 'gimmick', 'lessons' : 'lesson',  'losers' : 'loser', 'novels' : 'novel', 'reps' : 'rep', 'sharpshooters' : 'sharpshooter',  'whatnots' : 'whatnot', 'fanboys' : 'fanboy', 'destinations' : 'destination', 'morons' : 'moron',  'genderlocks' : 'genderlock', 'beasts' : 'beast', 'lengths' : 'length', 'owners' : 'owner', 'expressions' : 'expression',  'caveats' : 'caveat', 'riders' : 'rider', 'avatars' : 'avatar', 'doors' : 'door', 'hamsters' : 'hamster',  'shortcuts' : 'shortcut', 'suggestions' : 'suggestion', 'rifts' : 'rift', 'intentions' : 'intention',  'explosions' : 'explosion', 'innovations' : 'innovation', 'legends' : 'legend', 'cpus' : 'cpu', 'wardancers' : 'wardancer',  'skeletons' : 'skeleton', 'invisibles' : 'invisible', 'vms' : 'vm', 'particles' : 'particle', 'reskins' : 'reskin',  'roadmaps' : 'roadmap', 'furrys' : 'furry', 'trials' : 'trial', 'incentivizes' : 'incentivize', 'thresholds' : 'threshold',  'deadzones' : 'deadzone', 'beliefs' : 'belief', 'begginings' : 'beggining', 'pugs' : 'pug', 'outcomes' : 'outcome',  'raiders' : 'raider', 'timewalls' : 'timewall', 'legs' : 'leg', 'multitudes' : 'multitude', 'gits' : 'git',  'premades' : 'premade', 'behaviors' : 'behavior', 'onwards' : 'onward', 'drawbacks' : 'drawback', 'tryhards' : 'tryhard',  'vips' : 'vip', 'pixels' : 'pixel', 'lockouts' : 'lockout', 'cliffs' : 'cliff', 'moods' : 'mood', 'templates' : 'template',  'creatures' : 'creature', 'evolutions' : 'evolution', 'ideals' : 'ideal', 'afterthoughts' : 'afterthought',  'visions' : 'vision', 'macros' : 'macro', 'replacements' : 'replacement', 'diamonds' : 'diamond', 'los' : 'lo',  'experts' : 'expert', 'punishments' : 'punishment', 'defenses' : 'defense', 'dealers' : 'dealer', 'immortals' : 'immortal',  'inners' : 'inner', 'winrates' : 'winrate', 'buddys' : 'buddy', 'andys' : 'andy', 'wannabes' : 'wannabe',  'themeparks' : 'themepark', 'customs' : 'custom', 'tropes' : 'trope', 'judgements' : 'judgement', 'sidequests' : 'sidequest',  'launchers' : 'launcher', 'gms' : 'gm', 'episodes' : 'episode', 'placements' : 'placement', 'leaders' : 'leader',  'principles' : 'principle', 'upsides' : 'upside', 'streets' : 'street', 'marriages' : 'marriage', 'thematics' : 'thematic',  'potatoes' : 'potatoe', 'mechanisms' : 'mechanism', 'baldurs' : 'baldur', 'insights' : 'insight', 'hubs' : 'hub',  'modifications' : 'modification', 'leaderboards' : 'leaderboard', 'animals' : 'animal', 'penaltys' : 'penalty',  'popups' : 'popup', 'liners' : 'liner', 'hotkeys' : 'hotkey', 'widescreens' : 'widescreen', 'facets' : 'facet',  'announcements' : 'announcement', 'reductions' : 'reduction', 'farmers' : 'farmer', 'speedups' : 'speedup',  'norms' : 'norm', 'enjoyers' : 'enjoyer', 'moms' : 'mom', 'limitations' : 'limitation', 'workdays' : 'workday',  'spoilers' : 'spoiler', 'spenders' : 'spender', 'monks' : 'monk', 'stances' : 'stance', 'protagonists' : 'protagonist',  'workers' : 'worker', 'pinatas' : 'pinata', 'supporters' : 'supporter', 'fists' : 'fist', 'imbalances' : 'imbalance',  'miracles' : 'miracle', 'ips' : 'ip', 'robots' : 'robot', 'professions' : 'profession', 'agendas' : 'agenda',  'specializations' : 'specialization', 'vids' : 'vid', 'babes' : 'babe', 'revolutions' : 'revolution',  'hostages' : 'hostage', 'obstacles' : 'obstacle', 'villains' : 'villain', 'fps' : 'fp', 'components' : 'component',  'spreadsheets' : 'spreadsheet', 'arrows' : 'arrow', 'sailors' : 'sailor', 'maxers' : 'maxer', 'goldsellers' : 'goldseller',  'fundamentals' : 'fundamental', 'bastards' : 'bastard', 'videogames' : 'videogame', 'virtues' : 'virtue',  'indications' : 'indication', 'voiceovers' : 'voiceover', 'ripoffs' : 'ripoff', 'intents' : 'intent',  'spectrums' : 'spectrum', 'actors' : 'actor', 'consequences' : 'consequence', 'amateurs' : 'amateur',  'distractions' : 'distraction', 'corridors' : 'corridor', 'logos' : 'logo', 'woods' : 'wood', 'devices' : 'device',  'mons' : 'mon', 'digits' : 'digit', 'downfalls' : 'downfall', 'innocents' : 'innocent', 'sss' : 'ss', 'cells' : 'cell',  'grandmas' : 'grandma', 'framerates' : 'framerate', 'stunlocks' : 'stunlock', 'ammounts' : 'ammount',  'operations' : 'operation', 'angels' : 'angel', 'configurations' : 'configuration', 'loadouts' : 'loadout',  'mechs' : 'mech', 'wikis' : 'wiki', 'pies' : 'pie', 'bis' : 'bi', 'teammates' : 'teammate', 'obligations' : 'obligation',  'headaches' : 'headache', 'teenagers' : 'teenager', 'viewpoints' : 'viewpoint', 'randos' : 'rando',  'terrains' : 'terrain', 'als' : 'al', 'victims' : 'victim', 'chads' : 'chad', 'mommys' : 'mommy', 'fovs' : 'fov',  'liars' : 'liar', 'milestones' : 'milestone', 'lifeskills' : 'lifeskill', 'nfts' : 'nft', 'teens' : 'teen',  'hoverboards' : 'hoverboard', 'compositions' : 'composition', 'tienes' : 'tiene', 'wizards' : 'wizard',  'displeasures' : 'displeasure', 'corporations' : 'corporation', 'archers' : 'archer', 'debuffs' : 'debuff',  'ress' : 'res', 'standarts' : 'standart', 'restrictions' : 'restriction', 'keybindings' : 'keybinding',  'runners' : 'runner', 'writers' : 'writer', 'preparations' : 'preparation', 'tales' : 'tale', 'makeups' : 'makeup',  'taks' : 'tak', 'dolls' : 'doll', 'descriptions' : 'description', 'tournaments' : 'tournament',  'exclamations' : 'exclamation', 'guitars' : 'guitar', 'assasins' : 'assasin', 'snowflakes' : 'snowflake',  'nerfs' : 'nerf', 'zombies' : 'zombie', 'chapters' : 'chapter', 'bastantes' : 'bastante', 'makers' : 'maker',  'variables' : 'variable', 'ogs' : 'og', 'recipes' : 'recipe', 'guests' : 'guest', 'clusterfucks' : 'clusterfuck',  'drivers' : 'driver', 'dgs' : 'dg', 'lions' : 'lion', 'roadblocks' : 'roadblock', 'racists' : 'racist', 'relics' : 'relic',  'confirmations' : 'confirmation', 'todos' : 'todo', 'miles' : 'mile', 'commercials' : 'commercial', 'activations' : 'activation',  'tradeskills' : 'tradeskill', 'origins' : 'origin', 'competitors' : 'competitor', 'barrens' : 'barren', 'apples' : 'apple',  'payers' : 'payer', 'spammers' : 'spammer', 'dugeons' : 'dugeon', 'photos' : 'photo', 'assumptions' : 'assumption',  'horrors' : 'horror', 'clans' : 'clan', 'employees' : 'employee', 'simps' : 'simp', 'shooters' : 'shooter',  'hackers' : 'hacker', 'junkies' : 'junkie', 'mangas' : 'manga', 'changers' : 'changer', 'queus' : 'queu',  'gibs' : 'gib', 'traders' : 'trader', 'decorations' : 'decoration', 'critics' : 'critic', 'bossfights' : 'bossfight',  'freebies' : 'freebie', 'palettes' : 'palette', 'basements' : 'basement', 'viewers' : 'viewer', 'helmets' : 'helmet',  'nuances' : 'nuance', 'checkpoints' : 'checkpoint', 'nerds' : 'nerd', 'warlords' : 'warlord', 'brawlers' : 'brawler',  'lbs' : 'lb', 'advertisements' : 'advertisement', 'mmogs' : 'mmog', 'morals' : 'moral', 'nerias' : 'neria', 'lanes' : 'lane',  'isles' : 'isle', 'readers' : 'reader', 'algorithms' : 'algorithm', 'testers' : 'tester', 'folks' : 'folk',  'gladiators' : 'gladiator', 'emotions' : 'emotion', 'mods' : 'mod', 'scammers' : 'scammer', 'happends' : 'happend',  'enthusiasts' : 'enthusiast', 'reminders' : 'reminder', 'ots' : 'ot', 'laws' : 'law', 'loss' : 'los',  'neets' : 'neet', 'reviewers' : 'reviewer', 'comps' : 'comp', 'traits' : 'trait', 'maniacs' : 'maniac', 'meteors' : 'meteor',  'pills' : 'pill', 'collectathons' : 'collectathon', 'shards' : 'shard', 'expeditions' : 'expedition', 'healthbars' : 'healthbar',  'elitists' : 'elitist', 'temps' : 'temp', 'yrs' : 'yr', 'betrayals' : 'betrayal', 'conteúdos' : 'conteúdo',  'pontos' : 'ponto', 'movesets' : 'moveset', 'collectables' : 'collectable', 'phds' : 'phd', 'waypoints' : 'waypoint',  'lulzeks' : 'lulzek', 'relations' : 'relation', 'arses' : 'arse', 'incels' : 'incel', 'wonderlands' : 'wonderland',  'cheaters' : 'cheater', 'removals' : 'removal', 'osts' : 'ost', 'downwards' : 'downward', 'gaurdians' : 'gaurdian',  'sliders' : 'slider', 'nations' : 'nation', 'feats' : 'feat', 'gloves' : 'glove', 'outages' : 'outage', 'detriments' : 'detriment',  'counterparts' : 'counterpart', 'teles' : 'tele', 'resurrections' : 'resurrection', 'influencers' : 'influencer',  'mainquests' : 'mainquest', 'negativos' : 'negativo', 'cathedrals' : 'cathedral', 'barebones' : 'barebone',  'gamemodes' : 'gamemode', 'barbies' : 'barbie', 'fas' : 'fa', 'feitas' : 'feita', 'snoozefests' : 'snoozefest',  'youtubers' : 'youtuber', 'lasers' : 'laser', 'olympics' : 'olympic', 'peasants' : 'peasant', 'rangers' : 'ranger',  'errands' : 'errand', 'neurons' : 'neuron', 'unlockables' : 'unlockable', 'strangers' : 'stranger', 'friendships' : 'friendship',  'miniguns' : 'minigun', 'newborns' : 'newborn', 'toasters' : 'toaster', 'clichés' : 'cliché', 'dms' : 'dm',  'grievances' : 'grievance', 'estates' : 'estate', 'gcds' : 'gcd', 'buyers' : 'buyer', 'macguffins' : 'macguffin',  'minions' : 'minion', 'designers' : 'designer', 'ults' : 'ult', 'camels' : 'camel', 'fonts' : 'font',  'piñatas' : 'piñata', 'hairstyles' : 'hairstyle', 'citizens' : 'citizen', 'windmills' : 'windmill',  'temptations' : 'temptation', 'bandits' : 'bandit', 'implications' : 'implication', 'pois' : 'poi',  'typos' : 'typo', 'firewalls' : 'firewall', 'excrements' : 'excrement', 'fireballs' : 'fireball',  'trillions' : 'trillion', 'ears' : 'ear', 'attachments' : 'attachment', 'lifers' : 'lifer',  'payouts' : 'payout', 'clarifications' : 'clarification', 'factions' : 'faction', 'tangents' : 'tangent',  'eyebrows' : 'eyebrow', 'vistas' : 'vista', 'masochists' : 'masochist', 'indoors' : 'indoor', 'hallmarks' : 'hallmark',  'statistics' : 'statistic', 'edgelords' : 'edgelord', 'necromancers' : 'necromancer', 'archetypes' : 'archetype',  'grams' : 'gram', 'pistols' : 'pistol', 'tooltips' : 'tooltip', 'yards' : 'yard', 'artifacts' : 'artifact',  'injections' : 'injection', 'valleys' : 'valley', 'bloodstones' : 'bloodstone', 'indicators' : 'indicator',  'ressources' : 'ressource', 'swipers' : 'swiper', 'setpieces' : 'setpiece', 'economics' : 'economic',  'troves' : 'trove', 'elixirs' : 'elixir', 'toughts' : 'tought', 'gigs' : 'gig', 'teaspoons' : 'teaspoon',  'hallways' : 'hallway', 'screenshots' : 'screenshot', 'gamblers' : 'gambler', 'residents' : 'resident',  'mofos' : 'mofo', 'bananas' : 'banana', 'ragdolls' : 'ragdoll', 'abs' : 'ab', 'megaservers' : 'megaserver',  'translators' : 'translator', 'characteristics' : 'characteristic', 'tookis' : 'tooki', 'tablets' : 'tablet',  'strats' : 'strat', 'graders' : 'grader', 'thons' : 'thon', 'agents' : 'agent', 'nes' : 'ne', 'units' : 'unit',  'afterwards' : 'afterward', 'rookies' : 'rookie', 'christians' : 'christian', 'knockoffs' : 'knockoff',  'posters' : 'poster', 'elevators' : 'elevator', 'trackers' : 'tracker', 'techniques' : 'technique',  'cms' : 'cm', 'tactics' : 'tactic', 'baddies' : 'baddie', 'coasters' : 'coaster', 'leapstones' : 'leapstone',  'refugees' : 'refugee', 'setbacks' : 'setback', 'lips' : 'lip', 'mechas' : 'mecha', 'skillsets' : 'skillset',  'passages' : 'passage', 'defences' : 'defence', 'bonitos' : 'bonito', 'interessantes' : 'interessante',  'incidents' : 'incident', 'rituals' : 'ritual', 'quotas' : 'quota', 'characterizations' : 'characterization',  'amazonians' : 'amazonian', 'goblins' : 'goblin', 'golems' : 'golem', 'barbarians' : 'barbarian',  'defenders' : 'defender', 'hitpoints' : 'hitpoint', 'coupons' : 'coupon', 'equipamentos' : 'equipamento',  'laggs' : 'lagg', 'religions' : 'religion', 'willis' : 'willi', 'conferences' : 'conference', 'starfragments' : 'starfragment',  'hitlers' : 'hitler', 'spawners' : 'spawner', 'junglers' : 'jungler',  'vegetables' : 'vegetable', 'disconnections' : 'disconnection', 'dolars' : 'dolar', 'fireworks' : 'firework',  'bestimmtes' : 'bestimmte', 'flamethrowers' : 'flamethrower', 'skillshots' : 'skillshot', 'herbs' : 'herb',  'conventions' : 'convention', 'notions' : 'notion', 'ells' : 'ell', 'thigs' : 'thig',  'suckers' : 'sucker', 'daggers' : 'dagger', 'shineys' : 'shiney', 'quids' : 'quid', 'fricks' : 'frick',  'weekdays' : 'weekday', 'categories' : 'categorie', 'knees' : 'knee', 'beserkers' : 'beserker',  'misconceptions' : 'misconception', 'addons' : 'addon', 'parasites' : 'parasite', 'distinctions' : 'distinction',  'offenders' : 'offender', 'muppets' : 'muppet', 'milions' : 'milion', 'foudners' : 'foudner', 'hoarders' : 'hoarder',  'alliances' : 'alliance', 'crimes' : 'crime', 'organizations' : 'organization', 'always' : 'alway',  'autocompletes' : 'autocomplete', 'nowadays' : 'nowaday', 'peons' : 'peon', 'skillpoints' : 'skillpoint',  'lawsuits' : 'lawsuit', 'lads' : 'lad', 'fangs' : 'fang', 'balanceadas' : 'balanceada', 'gráficos' : 'gráfico',  'efeitos' : 'efeito', 'preços' : 'preço', 'consistentes' : 'consistente', 'motivos' : 'motivo', 'homies' : 'homie',  'friendlists' : 'friendlist', 'bazookas' : 'bazooka', 'prerequisites' : 'prerequisite', 'attemps' : 'attemp',  'pianos' : 'piano', 'psychopaths' : 'psychopath', 'interruptions' : 'interruption', 'puedes' : 'puede',  'horas' : 'hora', 'personajes' : 'personaje', 'upwards' : 'upward', 'recolors' : 'recolor', 'ants' : 'ant',  'founderpacks' : 'founderpack', 'mecânicas' : 'mecânica', 'atrasados' : 'atrasado',  'feminists' : 'feminist', 'mehanics' : 'mehanic', 'personalisations' : 'personalisation',  'ancestors' : 'ancestor', 'robotics' : 'robotic', 'sentiments' : 'sentiment', 'boes' : 'boe', 'stans' : 'stan',  'tards' : 'tard', 'crafters' : 'crafter', 'boars' : 'boar', 'distributors' : 'distributor',  'pelas' : 'pela', 'microstutters' : 'microstutter', 'plebs' : 'pleb', 'kmmorpgs' : 'kmmorpg',  'savers' : 'saver', 'cigarettes' : 'cigarette', 'docs' : 'doc', 'colleagues' : 'colleague',  'conteudos' : 'conteudo', 'vegas' : 'vega', 'achievments' : 'achievment', 'creditcards' : 'creditcard',  'realeases' : 'realease', 'rollouts' : 'rollout', 'asherons' : 'asheron', 'assets' : 'asset',  'avenues' : 'avenue', 'infinitos' : 'infinito', 'husbandos' : 'husbando', 'glaciers' : 'glacier',  'donations' : 'donation', 'positionals' : 'positional', 'speedhacks' : 'speedhack', 'charecters' : 'charecter',  'grindwalls' : 'grindwall', 'personas' : 'persona', 'milkers' : 'milker', 'coomers' : 'coomer',  'lootables' : 'lootable', 'yeets' : 'yeet', 'widths' : 'width', 'mismos' : 'mismo', 'bidders' : 'bidder',  'assholes' : 'asshole', 'overlords' : 'overlord', 'minuets' : 'minuet', 'vikings' : 'viking',  'npss' : 'nps', 'skillers' : 'skiller', 'workarounds' : 'workaround', 'dias' : 'dia', 'hookers' : 'hooker',  'possess' : 'posses', 'conclussions' : 'conclussion', 'corpses' : 'corpse', 'stairs' : 'stair',  'gauntlets' : 'gauntlet', 'fanatics' : 'fanatic', 'gienahs' : 'gienah', 'droprates' : 'droprate',  'stories' : 'storie', 'halos' : 'halo', 'ags' : 'ag', 'guildmates' : 'guildmate', 'convienences' : 'convienence',  'motives' : 'motive', 'jocks' : 'jock', 'neckbeards' : 'neckbeard', 'weirdos' : 'weirdo', 'survivors' : 'survivor',  'schoolers' : 'schooler', 'pathways' : 'pathway', 'stilettos' : 'stiletto', 'monarchs' : 'monarch', 'ness' : 'nes',  'series' : 'serie', 'haters' : 'hater', 'abilities' : 'abilitie', 'altas' : 'alta',  'dialouges' : 'dialouge', 'keywords' : 'keyword', 'complainers' : 'complainer', 'costumers' : 'costumer',  'grafics' : 'grafic', 'promotions' : 'promotion', 'execs' : 'exec', 'specialists' : 'specialist',  'alas' : 'ala', 'elves' : 'elve', 'access' : 'acces', 'hobbies' : 'hobbie', 'lootboxes' : 'lootboxe',  'pauldrons' : 'pauldron', 'scientists' : 'scientist', 'watchers' : 'watcher', 'autoattacks' : 'autoattack',  'matchups' : 'matchup', 'bos' : 'bo', 'dailies' : 'dailie', 'predictions' : 'prediction', 'degens' : 'degen',  'pubs' : 'pub', 'revelations' : 'revelation', 'instructions' : 'instruction', 'villagers' : 'villager',  'whims' : 'whim', 'possessions' : 'possession', 'ios' : 'io', 'gals' : 'gal', 'contributions' : 'contribution',  'backwards' : 'backward', 'shutdowns' : 'shutdown', 'pillars' : 'pillar', 'gandalfs' : 'gandalf', 'turrets' : 'turret',  'nps' : 'np', 'marines' : 'marine', 'playlists' : 'playlist', 'adjustments' : 'adjustment', 'modifiers' : 'modifier',  'newcomers' : 'newcomer', 'medals' : 'medal', 'penguins' : 'penguin', 'foes' : 'foe', 'admins' : 'admin',  'seizures' : 'seizure', 'casters' : 'caster', 'dums' : 'dum', 'variants' : 'variant', 'brownies' : 'brownie',  'tablespoons' : 'tablespoon', 'ribbons' : 'ribbon', 'destructibles' : 'destructible', 'observations' : 'observation',  'inquisitors' : 'inquisitor', 'rts' : 'rt', 'touchdowns' : 'touchdown', 'monies' : 'money', 'currencies' : 'currency',  'enemies' : 'enemy', 'activities' : 'activity', 'weeklies' : 'weekly', 'countries' : 'country', 'buddies' : 'buddy',  'cities' : 'city', 'qualities' : 'quality', 'crybabies' : 'crybaby', 'communities' : 'community', 'ladies' : 'lady',  'possibilities' : 'possibility', 'opportunities' : 'opportunity', 'memories' : 'memory', 'accessories' : 'accessory',  'similarities' : 'similarity', 'capabilities' : 'capability', 'identities' : 'identity', 'titties' : 'titty',  'armies' : 'army', 'policies' : 'policy', 'difficulties' : 'difficulty', 'varieties' : 'variety', 'families' : 'family',  'responsibilities' : 'responsibility', 'entries' : 'entry', 'quantities' : 'quantity', 'synergies' : 'synergy',  'inconsistencies' : 'inconsistency', 'boundaries' : 'boundary', 'capacities' : 'capacity', 'properties' : 'property',  'nationalities' : 'nationality', 'controversies' : 'controversy', 'apologies' : 'apology', 'economies' : 'economy',  'priorities' : 'priority', 'ideologies' : 'ideology', 'intricacies' : 'intricacy', 'bitties' : 'bitty',  'personalities' : 'personality', 'pennies' : 'penny', 'melodies' : 'melody', 'daylies' : 'dayly',  'discoveries' : 'discovery', 'fairies' : 'fairy', 'legendaries' : 'legendary', 'shinies' : 'shiny',  'availabilities' : 'availability', 'rarities' : 'rarity', 'duties' : 'duty', 'novelties' : 'novelty',  'functionalities' : 'functionality', 'affinities' : 'affinity', 'probabilities' : 'probability', 'minorities' : 'minority',  'idiocies' : 'idiocy', 'necessities' : 'necessity', 'centuries' : 'century', 'pussies' : 'pussy', 'theories' : 'theory',  'victories' : 'victory', 'mysteries' : 'mystery', 'backstories' : 'backstory', 'penalties' : 'penalty', 'authorities' : 'authority',  'furries' : 'furry', 'discrepancies' : 'discrepancy', 'harmonies' : 'harmony', 'frequencies' : 'frequency', 'fallacies' : 'fallacy',  'technologies' : 'technology', 'entities' : 'entity', 'strategies' : 'strategy', 'tendencies' : 'tendency',  'playabilities' : 'playability', 'monstrosities' : 'monstrosity', 'sensitivities' : 'sensitivity', 'bounties' : 'bounty',  'sceneries' : 'scenery', 'twenties' : 'twenty', 'thirties' : 'thirty', 'thingies' : 'thingy', 'bunnies' : 'bunny',  'batteries' : 'battery', 'deities' : 'deity', 'groceries' : 'grocery', 'abilities' : 'ability', 'abilitie' : 'ability', 'paypiggies' : 'paypiggy', 'goodies' : 'goody', 'tiddies' : 'tiddy', 'salaries' : 'salary', 'species' : 'specy', 'guildies' : 'guildy', 'galaxies' : 'galaxy', 'panties' : 'panty', 'deficiencies' : 'deficiency', 'normies' : 'normy', 'munnies' : 'munny', 'fundies' : 'fundy', 'teletubbies' : 'teletubby', 'cuties' : 'cuty', 'dalies' : 'daly', 'sidestories' : 'sidestory', 'activies' : 'activy', 'eggies' : 'eggy', 'glossaries' : 'glossary', 'norities' : 'nority', 'redundancies' : 'redundancy', 'shities' : 'shity', 'tinglies' : 'tingly', 'screenies' : 'screeny', 'responsabilities' : 'responsability', 'acitivities' : 'acitivity', 'specialities' : 'speciality', 'minihistories' : 'minihistory', 'incentivies' : 'incentivy', 'ssies' : 'ssy', 'curencies' : 'curency', 'ennemies' : 'ennemy', 'rivalries' : 'rivalry', 'experiencies' : 'experiency', 'sensibilities' : 'sensibility', 'poggies' : 'poggy', 'contemporaries' : 'contemporary', 'horrorstories' : 'horrorstory', 'boobies' : 'booby', 'ideaologies' : 'ideaology', 'inquiries' : 'inquiry', 'underpanties' : 'underpanty', 'patties' : 'patty', 'ablities' : 'ablity', 'similiarities' : 'similiarity'  }
    
goods = ['hell', 'hate', 'hype','super',
         'good','awful', 'best', 'great', 'nice', 'bad','awesome','cool','well','shit','worst' ]  
         # 'terrible', 'pretty','amaze', 'garbage'
        
changes = {'scenario' : 'story', 'mechanics': 'mechanic','mmos':'mmo','items':'item', 'money':'cash', 'bots':'bot','buy': 'purchase', 'astound': 'amaze','astonish': 'amaze', 'na': 'usa', 'russian':'russia', 'ru':'russia', 'european': 'europe', 'eu':'europe','anime': 'cutscene', 'dev': 'developer', 'bug':'error', 'hr':'hour', 'gud' : 'good', 'que' : 'queue', 'lvl' : 'level', 'fuck' : 'bad', 'suck' : 'bad', 'skills' : 'skill', 'grindy': 'grind', 'hone': 'grind','kr':'korea','korean':'korea'}
           # 'us': 'usa', 
         
#notaglist = ['JJR','JJS','JJ', 'RB', 'RBR', 'RBS']

notaglist = []

stopwordss = stopwords + stops + goods
         
for i in stops:
    stopwordss.append(i+'s')
         
stopwordss = list(set(stopwordss))

In [10]:
reviews = df['Review'].tolist()
stemmed = []
lemmatized = []


for line in tqdm(reviews):
    line_stem = []
    line_lemmatize = []
    line_JJRB = []
    line_token = nltk.pos_tag(line.split())
    for token, tag in line_token:
        if not tag in notaglist:
            if not token in stopwordss:
                if len(token) >= 2:
                    #st = stemmer(token)
                    le = lemmatizer(token,pos='v')

                    for times in range(5):
                        #if st in synonyms.keys():
                            #st = synonyms[st]
                        if le in synonyms.keys():
                            le = synonyms[le]

                    #if not st in stopwordss:
                        #if st in changes.keys():
                        #    line_stem.append(changes[st])
                        #else:
                        #    line_stem.append(st)

                    if not le in stopwordss:
                        if le in changes.keys():
                            line_lemmatize.append(changes[le])  
                        else:
                            line_lemmatize.append(le)  
                            
    #stemmed.append(line_stem)
    lemmatized.append(line_lemmatize)



100%|███████████████████████████████████████████████████████████████████████████| 85163/85163 [06:01<00:00, 235.81it/s]


In [11]:
lemmatized_nostopword = []


for i in tqdm(lemmatized):
    x = []
    for j in i:
        if not j in stopwordss:
            x.append(j)
    lemmatized_nostopword.append(x)

    
    
#df['Stemmed'] = stemmed
df['Lemmatized'] = lemmatized_nostopword

nums = []
for i in df['Lemmatized']:
    nums.append(len(i))
df['nums'] = nums
df = df.drop(df[df['nums'] == 0].index)


df = df.reindex(columns=['Review','Lemmatized',#'Stemmed',
                         'Positive=1','Create-Date','Up-Date','Purchase','Playtime','language','UserID','Cursor'])


df.to_pickle('stemming_data(새시작).pkl')
df.to_csv('stemming_data(새시작).csv', index= False)


100%|██████████████████████████████████████████████████████████████████████████| 85163/85163 [00:56<00:00, 1518.32it/s]


# Lemmatized 성능이 더 좋은듯?

In [12]:
#print('Raw : ')
#print(df['Review'][0] + "\n")
#print('Lemmatized : ')
#print(' '+' '.join(df['Lemmatized'][0])+ "\n")
#print('Stemmed : ')
#print(' '+' '.join(df['Stemmed'][0])+ "\n")

# Freq

In [13]:
df = pd.read_pickle('stemming_data(새시작).pkl')
Lemmatized = df['Lemmatized'].to_list()

In [14]:
freq = dict()
for line in Lemmatized:
    for token in line:
        freq[f"{token}"] = 0
        
for line in Lemmatized:
    for token in line:
        freq[f"{token}"] += 1
freq = sorted(freq.items(), key=(lambda x: x[1]),reverse=True)     
Freqdf = pd.DataFrame.from_dict(freq)
Freqdf.columns = ['Words','Freq']
Freqdf.to_csv('Freq(새시작).csv',index=False)

# TFIDF

In [15]:
DATA = df
list_N = []
for i in DATA['Lemmatized']:
    word = ' '
    for j in i:
        word += j
        word += ' '
    list_N.append(word)
    
TFIDF = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',min_df = 2).fit(list_N)
x = TFIDF.transform(list_N).toarray()
y = sorted(TFIDF.vocabulary_)
TFIDFdf = pd.DataFrame(x)
z = list(TFIDFdf.sum())
TFIDFdf = pd.DataFrame(list(zip(y,z)),columns=['Words','TF-IDF']).sort_values(by=['TF-IDF'], axis=0, ascending=False)
TFIDFdf.to_csv('TFIDF(새시작).csv',index=False)

# MERGE ( TFIDF & Freq )

In [16]:
merge = pd.merge(TFIDFdf,Freqdf, how='outer',on='Words').sort_values(by=['TF-IDF'], axis=0, ascending=False)
merge.to_csv('TFIDF&Freq(새시작).csv',index=False)
merge[merge['TF-IDF'] >1].to_csv('TFIDF&Freq(over 1)(새시작).csv',index=False)
merge.iloc[:400].to_csv('TFIDF&Freq(~400)(새시작).csv',index=False)

# STM 전처리

In [17]:
df = pd.read_pickle('stemming_data(새시작).pkl')

UP = df['Up-Date'].to_list()
DD = []
time1 = datetime(2022, 2, 12)
for i in UP:
    Y = int(i[0:4])
    M = int(i[5:7])
    D = int(i[8:10])
    time2 = datetime(Y, M, D)
    #DD.append(time1+timedelta(days=int((time2-time1).days/7)*7))
    #DD.append(int(((time2-time1).days/7)+1)*7)
    DD.append(int((time2-time1).days)+1)
df['Day'] = DD

recommend = df['Up-Date'].to_list()
recommend = df['Positive=1'].to_list()
rec = [] 
for i in recommend:
    if i == 1:
        rec.append("Recommend")
    else:
        rec.append("Not Recommend")

df['Recommend'] = rec

be = len(df)

list_N = []
wordnumber = []
wordN = []
for i in df['Lemmatized']:
    word = ' '
    cnt = 0
    for j in i:
        cnt += 1
        word += j
        word += ' '
    wordN.append(len(set(i)))
    list_N.append(word)
    wordnumber.append(cnt)

df['line'] = list_N
df['num'] = wordnumber
df['wordN'] = wordN
df = df[df['wordN'] > 3]
df = df[df['num'] > 5]

af = len(df)

STMdf = df.reindex(columns=['line', 'Day', 'Recommend'])

STMdf = STMdf.drop_duplicates('line')

STMdf = STMdf.sort_values(by=['line'])

STMdf.to_csv('./STM/STMs(새시작).csv',index = False)




print(f'사라진데이터 = {be - af}개\n')
print(f'총    데이터 = {af}개')


사라진데이터 = 38095개

총    데이터 = 33474개


# DTM (???)